In [11]:
!pip3 install dash

     |████████████████████████████████| 1.1 MB 31.3 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.7 MB 65.3 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 6.8 MB/s  eta 0:00:01
     |████████████████████████████████| 1.8 MB 69.6 MB/s eta 0:00:01
     |████████████████████████████████| 288 kB 64.7 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 66.4 MB/s eta 0:00:01
  Created wheel for dash: filename=dash-1.21.0-py3-none-any.whl size=1100478 sha256=a61fab4e60c1ead7823cd4f1b7a5a8915703cf86b785140d2262e8b0655c9b08
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/56/e4/21/8a1190a7c31f3aa3f8834b05126ded8bf105b8fd11fad1806e
  Created wheel for dash-core-components: filename=dash_core_components-1.17.1-py3-none-any.whl size=3738983 sha256=6009da8bced541a49e37578d921eb60541cfd4c4b20960f0e7422f9b7b506554
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/08/ae/6a/0f4a0ee8fa216e

In [12]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [13]:
spacex_launch = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(spacex_launch)

In [14]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [15]:
all=spacex_df.groupby(['Launch Site'])['class'].count().reset_index()
all

,Launch Site,class
0,CCAFS LC-40,26
1,CCAFS SLC-40,7
2,KSC LC-39A,13
3,VAFB SLC-4E,10


In [16]:
a=list(spacex_df['Launch Site'].unique())
a.append('ALL sites')
a

['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40', 'ALL sites']

In [17]:
launch_site={a[4]:'OPT1', a[0]:'OPT2', a[1]:'OPT3', a[2]:'OPT4', a[3]:'OPT5'}
launch_site

{'ALL sites': 'OPT1',
 'CCAFS LC-40': 'OPT2',
 'VAFB SLC-4E': 'OPT3',
 'KSC LC-39A': 'OPT4',
 'CCAFS SLC-40': 'OPT5'}

In [18]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [19]:
# Create a dash application
app = dash.Dash(__name__)

In [ ]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                html.Div([
                                    html.Div(
                                        [
                                        html.H2('Launch Site:', style={'margin-right': '2em'}),
                                        ]
                                    ),
                                        dcc.Dropdown(id='site-dropdown', options=[{'label': k, 'value': v} for k,v in launch_site.items()],
                                        value='OPT1',
                                        placeholder='Select a launch site',
                                        searchable = True,
                                        style={'width': '80%', 'padding': '3px', 'textAlign': 'center','font-size': 20}),
                                    ], style={'display': 'flex'}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                    dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000,
                                        value=[spacex_df['Payload Mass (kg)'].min(),spacex_df['Payload Mass (kg)'].max()],
                                        marks={i*1000: '{}'.format(1000 * i) for i in range(11)}),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
# Add computation to callback function and return graph
def get_graph(site):
    if site=='OPT1':
        all_pie=spacex_df.groupby(['Launch Site'])['class'].count().reset_index()
        fig = px.pie(all, values='class', names='Launch Site', title='Launch Sites with shares of launches')
    else:
        for k, v in launch_site.items():
            if v == site:
                ilk=spacex_df[spacex_df['Launch Site']==k]
                iki=ilk.groupby(['class'])['Launch Site'].count().reset_index()
                fig = px.pie(iki, values='Launch Site', names='class', title='Success Rate of Launch Site ')
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])
               
def get_scatter(site, payload):
    if site=='OPT1':
        spacex_df_payload=spacex_df[(spacex_df['Payload Mass (kg)']>=payload[0]) & (spacex_df['Payload Mass (kg)']<=payload[1])]
        fig = px.scatter(spacex_df_payload, x="Payload Mass (kg)", y="class", color="Booster Version Category")
    else:
        for k, v in launch_site.items():
            if v == site:
                spacex_df_payload=spacex_df[(spacex_df['Launch Site']==k)& (spacex_df['Payload Mass (kg)']>=payload[0]) & (spacex_df['Payload Mass (kg)']<=payload[1])]
                fig = px.scatter(spacex_df_payload, x="Payload Mass (kg)", y="class", color="Booster Version Category")
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
